In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Example user-item interaction matrix (replace with your data)
user_item_matrix = np.random.randint(0, 2, (100, 50))  # Binary interaction (implicit feedback)

# Convert to triplets: (user_id, item_id, interaction)
user_ids, item_ids = np.where(user_item_matrix > 0)
interactions = user_item_matrix[user_ids, item_ids]
data = pd.DataFrame({"user_id": user_ids, "item_id": item_ids, "interaction": interactions})

# Encode user and item IDs
user_mapping = {u: i for i, u in enumerate(data['user_id'].unique())}
item_mapping = {i: j for j, i in enumerate(data['item_id'].unique())}

data['user_id'] = data['user_id'].map(user_mapping)
data['item_id'] = data['item_id'].map(item_mapping)

# Train-test split
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)


In [4]:
display(user_item_matrix)
display(interactions)
display(data.head())
display(train_data.head())
display(val_data.head())

array([[0, 1, 0, ..., 0, 1, 1],
       [0, 1, 1, ..., 0, 1, 1],
       [1, 1, 1, ..., 0, 0, 1],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 1, 1, ..., 1, 0, 1]])

array([1, 1, 1, ..., 1, 1, 1])

,user_id,item_id,interaction
0,0,0,1
1,0,1,1
2,0,2,1
3,0,3,1
4,0,4,1


,user_id,item_id,interaction
1309,52,43,1
157,6,13,1
2367,93,37,1
789,32,9,1
2261,89,31,1


,user_id,item_id,interaction
1055,42,12,1
1703,67,35,1
194,8,23,1
965,39,21,1
1857,74,0,1


In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dot, Flatten, Dense

# Hyperparameters
num_users = len(user_mapping)
num_items = len(item_mapping)
embedding_dim = 50

# Model architecture
user_input = Input(shape=(1,), name="user_input")
item_input = Input(shape=(1,), name="item_input")

user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim, name="user_embedding")(user_input)
item_embedding = Embedding(input_dim=num_items, output_dim=embedding_dim, name="item_embedding")(item_input)

# Compute dot product (interaction score)
dot_product = Dot(axes=2, name="dot_product")([user_embedding, item_embedding])
dot_product = Flatten(name="flatten")(dot_product)

# Add a dense layer (optional, for complex interactions)
output = Dense(1, activation='sigmoid', name="output")(dot_product)

# Build the model
model = Model(inputs=[user_input, item_input], outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [6]:
from tensorflow.keras.callbacks import TensorBoard

# Set up TensorBoard logging
log_dir = "logs/recommendation_system"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1, embeddings_freq=1)

# Prepare inputs for training
train_users = train_data['user_id'].values
train_items = train_data['item_id'].values
train_interactions = train_data['interaction'].values

val_users = val_data['user_id'].values
val_items = val_data['item_id'].values
val_interactions = val_data['interaction'].values

# Train the model
history = model.fit(
    [train_users, train_items],
    train_interactions,
    validation_data=([val_users, val_items], val_interactions),
    epochs=10,
    batch_size=32,
    callbacks=[tensorboard_callback]
)

print(f"Run TensorBoard with: tensorboard --logdir {log_dir}")

Epoch 1/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.8882 - loss: 0.6853 - val_accuracy: 1.0000 - val_loss: 0.6617
Epoch 2/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.6519 - val_accuracy: 1.0000 - val_loss: 0.6258
Epoch 3/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 0.6069 - val_accuracy: 1.0000 - val_loss: 0.5415
Epoch 4/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.4876 - val_accuracy: 1.0000 - val_loss: 0.3300
Epoch 5/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.2594 - val_accuracy: 1.0000 - val_loss: 0.1336
Epoch 6/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.0995 - val_accuracy: 1.0000 - val_loss: 0.0567
Epoch 7/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0431 - val_accuracy: 1.0000 - val_loss: 0.0297
Epoch 8/10
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0232 - val_accuracy: 1.0000 - val_loss

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

In [8]:
# Evaluate the model
loss, accuracy = model.evaluate([val_users, val_items], val_interactions)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")

# Predict recommendations for a specific user
user_id = 0
items_to_score = np.arange(num_items)  # Score all items for the user
user_array = np.full(len(items_to_score), user_id)

predicted_scores = model.predict([user_array, items_to_score])
recommendations = np.argsort(-predicted_scores.flatten())[:10]  # Top 10 items
print(f"Top 10 recommendations for User {user_id}: {recommendations}")


16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 0.0089 
Validation Loss: 0.00883846078068018, Validation Accuracy: 1.0
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Top 10 recommendations for User 0: [34 38 40  0 42  2 41 14 20 18]
